## Plots with Chl-a for Hawaii bloom

In [1]:
# Import all the stuff!
import earthaccess
import xarray as xr
from xarray.backends.api import open_datatree
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib as mpl
from matplotlib.patches import Polygon
from matplotlib.path import Path
import cartopy.crs as ccrs
import numpy as np
import datetime
#The function defined just creates a list of dates between two dates of choice (in this case)
def date_range_list(start_date, end_date):
    # Return list of datetime.date objects (inclusive) between start_date and end_date (inclusive).
    date_list = []
    curr_date = start_date
    while curr_date <= end_date:
        date_list.append(curr_date)
        curr_date += datetime.timedelta(days=1)
    return date_list

In [2]:
# Define our timespan and search the data
tspan = ("2024-07-07", "2024-08-5")
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL_NRT",
    temporal=tspan,
    granule_name="*.DAY.*4km.*",
)

In [3]:
# Nest all images along the date (29 satellite images)
paths = earthaccess.open(results)
dataset = xr.open_mfdataset(
    paths,
    combine="nested",
    concat_dim="date",
)
# Print structure of the dataset
dataset

QUEUEING TASKS | :   0%|          | 0/30 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/30 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/30 [00:00<?, ?it/s]

<xarray.Dataset> Size: 4GB
Dimensions:  (date: 30, lat: 4320, lon: 8640, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat      (lat) float32 17kB 89.98 89.94 89.9 89.85 ... -89.9 -89.94 -89.98
  * lon      (lon) float32 35kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Dimensions without coordinates: date, rgb, eightbitcolor
Data variables:
    chlor_a  (date, lat, lon) float32 4GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    palette  (date, rgb, eightbitcolor) uint8 23kB dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/64)
    product_name:                      PACE_OCI.20240707.L3m.DAY.CHL.V2_0.chl...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/CHL/2.0
    keywords:                          Earth Science > Oceans > Ocean Chemist...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         3310866
    data_minimum:                      0.0009999999
    data_maximum:                      98.42797

In [4]:
# Slice the dataset to extract only the region that we want
dataset = dataset.sel(lat=slice(34.0, 22.0), lon=slice(-174.0, -142))



In [5]:
# extract chla data
chla = dataset["chlor_a"]
# plot the mean for the region
chla_avg = chla.mean("date")
chla_avg.attrs.update(
    {
        "long_name": chla.attrs["long_name"],
        "units": f'lg({chla.attrs["units"]})',
    }
)
# extract longitude and latitude
lon = dataset["lon"]
lat = dataset["lat"]

In [6]:
# calculate basic statistics for chla
chla_std = chla.std("date") #standard deviation
chla_min = chla.min("date") # minimum
chla_max = chla.max("date") # maximum
chla_amp = chla_max - chla_min # amplitude

In [7]:
# Plot the average Chla during the study period
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels={"left": "y", "bottom": "x"}, alpha=0.3)
f1 = ax.pcolormesh(lon, lat, chla_avg, norm=mpl.colors.LogNorm(vmin=0.01, vmax=.3))
cbar = plt.colorbar(f1, fraction=0.02)
cbar.set_label('Chla', fontsize=14)
filename = 'chl_mean_07July02August.png'
plt.savefig(filename,format = 'png', bbox_inches = 'tight', dpi = 300)
plt.close()

/srv/conda/envs/notebook/lib/python3.10/site-packages/dask/array/numpy_compat.py:56: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


In [8]:
# Plot the max Chla during the study period
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels={"left": "y", "bottom": "x"}, alpha=0.3)
ax.pcolormesh(lon, lat, chla_max, norm=mpl.colors.LogNorm(vmin=0.01, vmax=.5))
filename = 'chl_max_07July02August.png'
plt.savefig(filename,format = 'png', bbox_inches = 'tight', dpi = 300)
plt.close()

In [9]:
# Plot the standard deviation of Chla + a square around the core of the bloom
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels={"left": "y", "bottom": "x"}, alpha=0.3)
f1 = ax.pcolormesh(lon, lat, chla_std, cmap=plt.cm.Reds, norm=mpl.colors.LogNorm(vmin=0.01, vmax=.5))
cbar = plt.colorbar(f1, fraction=0.02)
cbar.set_label('Chla', fontsize=14)
ROI_verts = [(-157.5, 28), (-159, 28), (-159, 26), (-157.5, 26)]
poly_ROI = Polygon(list(ROI_verts), facecolor=[1,1,1,0], edgecolor='k', linewidth=2, linestyle='-', zorder=2)
plt.gca().add_patch(poly_ROI)
filename = 'chl_bloommarked.png'
plt.savefig(filename,format = 'png', bbox_inches = 'tight', dpi = 300)
plt.close()

In [10]:
# Create the gif with the daily images

# create a list of dates and the frames
frames = np.arange(np.size(chla, axis=0))
date_list_gif = date_range_list(datetime.datetime(2024,7,7), datetime.datetime(2024,8,5))
date_list = date_range_list(datetime.datetime(2024,7,7), datetime.datetime(2024,8,5))
fig = plt.figure()
def update(i):
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.gridlines(draw_labels={"left": "y", "bottom": "x"})
    ax.pcolormesh(lon, lat, chla[i,:,:],norm=mpl.colors.LogNorm(vmin=0.01, vmax=.3))
    poly_ROI = Polygon(list(ROI_verts), facecolor=[1,1,1,0], edgecolor='k', linewidth=2, linestyle='-', zorder=2)
    plt.gca().add_patch(poly_ROI)
    ax.text(-173, 23, date_list_gif[i].strftime('%Y-%m-%d'), weight='bold', fontsize=14)
    #ax.set_extent([-174.0, -142.0, 22.0, 34])
    return ax

an = animation.FuncAnimation(fig=fig, func=update, frames=frames, interval=500)
filename = f'chl_gif_07July02August.gif'
an.save(filename, writer="pillow", dpi = 300)
plt.close()

In [11]:
# Slice the dataset to extract only the region that we want
dataset = dataset.sel(lat=slice(28.0, 26.0), lon=slice(-159.0, -157.5))

In [12]:
# extract chla data
chla = dataset["chlor_a"]
# plot the mean for the region
chla_avg = chla.mean("date")
chla_avg.attrs.update(
    {
        "long_name": chla.attrs["long_name"],
        "units": f'lg({chla.attrs["units"]})',
    }
)
# extract longitude and latitude
lon = dataset["lon"]
lat = dataset["lat"]

In [13]:
# calculate basic statistics for chla
chla_std = chla.std("date") #standard deviation
chla_min = chla.min("date") # minimum
chla_max = chla.max("date") # maximum
chla_amp = chla_max - chla_min # amplitude

In [14]:
# Plot the average Chla during the study period
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels={"left": "y", "bottom": "x"}, alpha=0.3)
f1 = ax.pcolormesh(lon, lat, chla_avg, norm=mpl.colors.LogNorm(vmin=0.01, vmax=.3))
cbar = plt.colorbar(f1, fraction=0.02)
cbar.set_label('Chla', fontsize=14)
filename = 'chl_mean_07July02August_zoom.png'
plt.savefig(filename,format = 'png', bbox_inches = 'tight', dpi = 300)
plt.close()

In [15]:
## Average over region
chl_ROI_pixels = chla
chl_ROI_1D_mean = np.nanmean(chl_ROI_pixels, (1,2))


/tmp/ipykernel_1075/92779025.py:3: RuntimeWarning: Mean of empty slice
  chl_ROI_1D_mean = np.nanmean(chl_ROI_pixels, (1,2))


In [16]:
#Plot mean over time
fig, ax = plt.subplots()
ax.plot(date_list, chl_ROI_1D_mean, '-o')
ax.set_xticks(ticks=date_list, labels=date_list, rotation=90)
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%d/%m')
ax.xaxis.set_major_formatter(myFmt)
#ax.set_xticks(rotation=90)
ax.set_ylabel('Chlor_a')
ax.set_xlabel('Time')
filename = 'chl_timeseries_Hawaai.png'
plt.savefig(filename,format = 'png', bbox_inches = 'tight', dpi = 300)
plt.close()